In [1]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7
)

In [2]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_info}"),
    ]
)

In [3]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
)

In [4]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)

好的，讓我來對這家台北市中心的義大利餐廳進行專業的評論分析：

**餐廳名稱：** (請提供餐廳名稱，我才能更精準地評論)

**整體評價：** 這家餐廳在台北市中心提供了一份相對優質的傳統義式用餐體驗，但其高價位和較長的等待時間可能會讓一些消費者感到不便。總體而言，適合追求浪漫氛圍、重視用餐品質，且預算充足的顧客。

**優點：**

* **環境優雅：** 餐廳的優雅環境為用餐提供了良好的氛圍，尤其適合約會或特殊場合，能提升用餐的整體感受。
* **服務親切：** 服務人員的親切服務能讓顧客感到賓至如歸，提升用餐的舒適度。
* **食物品質不錯：**  這點很重要！雖然份量少，但食物本身品質良好，表明餐廳在食材選擇和烹飪技巧方面有所用心，能讓顧客對食物的品質有信心。
* **適合約會或特殊場合：**  餐廳的定位明確，為顧客提供了適合的用餐選擇，方便顧客進行選擇。

**缺點：**

* **價格偏高：**  高價位是最大的負擔。需要考量顧客的消費能力，並評估其與食物份量的搭配是否合理。
* **等待時間較長：**  這會影響顧客的用餐體驗，尤其是在用餐高峰期。餐廳需要優化排程，減少顧客等待時間。
* **份量偏少：**  雖然食物品質不錯，但份量偏少會讓顧客覺得物價不高，甚至會影響飽足感。建議餐廳可以考慮增加部分菜品的份量，或提供更豐富的選擇。

**建議：**

* **提升效率：** 餐廳可以考慮增加服務人員，或優化排程系統，以減少顧客等待時間。
* **菜單調整：**  針對高價位，可以考慮增加一些特色菜品或套餐，以提升價值感。同時，可以重新評估部分菜品的份量，增加顧客的飽足感。
* **營造氛圍：**  持續維持優雅的環境，並提供一些額外的服務（例如：免費的開瓶酒、生日驚喜等），以提升顧客的用餐體驗。

**總評：** 

這家餐廳的潛力很高，但需要針對其缺點進行改善，才能更好地滿足顧客的需求，並在競爭激烈的台北市中心市場中脫穎而出。 

**請提供餐廳名稱，我能更精準地分析，並加入更具體的評論內容！** 例如，可以提供菜單、價格、菜品描述等資訊。

